In [ ]:
%pip install -q numpy
%pip install -q scipy
%pip install -q matplotlib
%pip install -q scikit-learn
%pip install -q pandas
%pip install -q statsmodels

In [ ]:
import numpy as np
from numpy import ndarray

import scipy as sp
from scipy import stats

import matplotlib.pyplot as plt

import pandas as pd
from pandas import DataFrame

import statsmodels.api as sm

In [ ]:
from os import getcwd

CWD: str = getcwd()
ASSETS_DIR: str = CWD + "/Assets-2_2"
DATASET_DIR: str = ASSETS_DIR + "/heightWeight.csv"

# Questão A

In [ ]:
def linear_function(x: ndarray) -> ndarray:
    return x * 2 + 1

In [ ]:
def uniform_distribution(x: ndarray) -> ndarray:
    return np.random.uniform(size=len(x))

In [ ]:
X_POINTS: ndarray = np.linspace(0, 1, 9)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
ax1, ax2 = axes

# ---- Linear function ----
y1: ndarray = linear_function(X_POINTS)
ax1.scatter(X_POINTS, y1)

# get the linear regression
slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(X_POINTS, y1)
line1: ndarray = slope1 * X_POINTS + intercept1
ax1.plot(X_POINTS, line1, label="Linear regression", linestyle="--")

# ---- Uniform distribution ----
y2: ndarray = uniform_distribution(X_POINTS)
ax2.scatter(X_POINTS, y2)

# get the linear regression
slope2, intercept2, r_value2, p_value2, std_err2 = stats.linregress(X_POINTS, y2)
line2: ndarray = slope2 * X_POINTS + intercept2
ax2.plot(X_POINTS, line2, label="Linear regression", linestyle="--")

ax1.set_title("Linear function")
ax2.set_title("Uniform Distribution")

plt.plot()

# Questão B

## Valores para a visualização dos dados

In [ ]:
BINS: int = 40
DENSITY: bool = True

## Carregando o dataset de pesos e alturas do [Kaggle](https://www.kaggle.com/datasets/burnoutminer/heights-and-weights-dataset?resource=download)

Esse dataset contém o total de 25 mil amostras de diferentes humanos de 18 anos de idades. Suas colunas são `Height(Inches)` em polegadas (_inches_) e `Weight(Pounds)` em libras (_pounds_). Utilizaremos apenas a coluna de altura (`Height(Inches)`) em passaremos ela para centímetros para melhorar a visualização

In [ ]:
df: DataFrame = pd.read_csv(DATASET_DIR)

display(df)

## Manipulação dos dados

Iremos transformar a coluna `Height(Inches)` em valores mais comuns para o brasileiro, então converteremos os valores para centímetros e iremos trocar o nome da coluna.

Para conversão, basta multiplicar a quantidade de polegadas vezes 2.54. Também converteremos a coluna de `weight` para que ela esteja convertida para quilogramas, basta 
multiplicar cada amostra por 0.453592.

In [ ]:
HEIGHT_COLUMN: str = "height"
WEIGHT_COLUMN: str = "weight"

# create a copy, rename the Height(Inches) to height, return just the height column
df_cleaned: DataFrame = df.copy().rename(columns={"Height(Inches)":HEIGHT_COLUMN, "Weight(Pounds)":WEIGHT_COLUMN}).drop(columns=["Index"])

# convert to cm
df_cleaned[HEIGHT_COLUMN] = df_cleaned[HEIGHT_COLUMN].map(lambda x: x*2.54)

# convert to kg
df_cleaned[WEIGHT_COLUMN] = df_cleaned[WEIGHT_COLUMN].map(lambda x: x*0.453592)

display(df_cleaned)

## Análise da altura

Calculamos a média e o desvio padrão para conseguirmos encontrar o intervalo de confiança da distribuição normal. Para buscar esse intervalo 
utilizamos o `scipy.stats.norm.interval` para pegarmos exatamente o intervalo que desejamos.

A formula usada é:

$$
\text{CI} = \bar{x} \pm z \times \frac{\sigma}{\sqrt{n}}
$$

- $z$ - Nível de confiança
- $\bar{x}$ - Média da amostra
- $\sigma$ - Desvio padrão da amostra
- $n$ - Tamanho da amostra

In [ ]:
mean: float = df_cleaned[HEIGHT_COLUMN].mean()
std: float = df_cleaned[HEIGHT_COLUMN].std() # type: ignore

confidences: tuple[float, ...] = (0.85, 0.9, 0.95, 0.99)
results: dict[float, tuple] = {}

for confidence in confidences:
    interval: tuple = stats.norm.interval(confidence, loc=mean, scale=std)
    print(f"Para uma confiança de {int(confidence * 100)}%, temos o intervalo: [{interval[0]}, {interval[1]}]")
    results[confidence] = interval

## Criando o histograma para mostrar graficamente a distribuição

In [ ]:
colors: tuple[str, str, str, str] = ('#66b3ff', '#3399ff', '#0066cc', '#0044cc')

plt.figure(figsize=(10, 6))

# histogram
plt.hist(df_cleaned[HEIGHT_COLUMN], density=DENSITY, bins=BINS, alpha=0.3, color='grey', label="Dados (Amostra)")

# mean line
plt.axvline(mean, color="red", linestyle="--", label=f"Média: {mean:.5f}")

# confidence bar
y_max: float = plt.gca().get_ylim()[1] # graphic top
confidence_heights: tuple[float, float, float, float] = (y_max*0.1, y_max*0.2, y_max*0.3, y_max*0.4)

for i, (confidence, interval) in enumerate(results.items()):
    erro = (interval[1] - interval[0]) / 2

    plt.errorbar(mean, confidence_heights[i], xerr=erro, fmt="o", capsize=8, color=colors[i], label=f"Intervalo de Confiança: {int(confidence*100)}%")

plt.ylim(0, y_max * 1.2) # increase the graphic height
plt.title("Distribuição da Altura e Intervalo de Confiança")
plt.xlabel("Altura (cm)")
plt.ylabel("Densidade (número de amostras no bin)")
plt.legend()
plt.show()

# Questão C

## Definindo a quantidade de amostras para cada grupo

In [ ]:
SAMPLES_NUMBER: int = 100

## Criando os grupos com base no dataset da questão anterior

In [ ]:
generator = np.random.default_rng()

# get a random sample of values to control group
control = df_cleaned[HEIGHT_COLUMN].sample(SAMPLES_NUMBER, random_state=generator)

# get a random sample of values to test group
test = df_cleaned[HEIGHT_COLUMN].sample(SAMPLES_NUMBER, random_state=generator)

## Utilizando `scipy.stats.ttest_ind` para verificar o p_value e o t_stat, ambos provindos da fórmula estatística da distribuição T-Student

In [ ]:
t_stat, p_value = stats.ttest_ind(control, test)

print(f"Média Controle: {control.mean()}")
print(f"Média Teste: {test.mean()}")
print(f"Estatística T: {t_stat}")
print(f"P-value: {p_value}")

## Avaliação do p_value

Se o valor de p_value for menor que um dado $\alpha$ qualquer no intervalo $[0, 1]$, então a nossa hipótese nula está correta. Quantos mais valores aleatórios forem buscados, menor 
será o t_stat, o qual representa a diferença entre o grupo controle e o grupo de teste. Isso significa que mais dados dão uma confiança muito maior para os nossos resultados. Dado essa
explicação, decidi utilizar um valor de $\alpha$ igual a 0.05, para que o `p_value` seja menor que ele, ou seja, uma confiança maior que 95%.

Logo, a Hipótese Nula ($H_0$) é de que os dados que pegamos não são distintos. E a Hipótese Alternativa ($H_1$) é de que os dados são estatisticamente diferentes entre os grupos, ou 
seja, são diferentes comuns.

- $H_0$ - Os grupos são estatisticamente iguais
- $H_1$ - Os grupos são estatisticamente diferentes

Em outras palavras, o $\alpha$ é um valor de confiança que aceitamos tolerar. Caso seja um $\alpha$ muito grande, então a chance de ser por a caso é maior e, por isso, o resultado não
é considerado confiável. Desse modo, o `p_value` reflete a chance de termos conseguido esse resultado por a caso.

In [ ]:
alpha: float = 0.05

if p_value < alpha: # type: ignore
    print(f"Decisão: Hipótese nula rejeitada. O p-value é suficientemente confiável: {p_value}")
    print(f"Conclusão: Existe uma diferença estatisticamente significativa entre os grupos")
else:
    print(f"Decisão: Hipótese nula aceita. O p-value não é suficientemente confiável: {p_value}")
    print(f"Conclusão: Existe uma diferença estatisticamente pequena entre os grupos")

# Questão D

O slide pode ser encontrado no [link](https://docs.google.com/presentation/d/18qr6W9VnDn1slNsL1n7vQRvynUyGTF1Agm1K0FSbRpo/edit?usp=sharing)

# Questão E

## Calculando o valor de $t$ crítico para utilizar com a distribuição _t-student_

In [ ]:
from scipy.stats import t

confidence_level: float = 0.975
degrees_of_freedon: int = 49

X_sample: pd.Series = df_cleaned[HEIGHT_COLUMN].sample(50)
mean: float = X_sample.mean()
std: float = X_sample.std()

t_critical: np.float64 = t.ppf(confidence_level, degrees_of_freedon) # type: ignore
print(t_critical)

## Calculando o intervalo de confiança com base da fórmula

$$
\bar{x} \pm t^* \cdot \frac{s}{\sqrt{n}}
$$

- $\bar{x}$ - Média dos dados coletados
- $t^*$ - Valor dado pela distribuição _t-student_
- $s$ - Desvio padrão das amostras
- $n$ - Número de amostras

Por meio dessa fórmula é possível identificar qual é a chance o intervalo de confiança

In [ ]:
def t_variation(x: ndarray, t_critical: float) -> float:
    return t_critical * (x.std() / np.sqrt(len(x)))

In [ ]:
t_var: float = t_variation(X_sample.to_numpy(), t_critical)
interval: tuple[float, float] = (mean - t_var, mean + t_var)

print(" Os Intervalos obtidos ".center(60, "-"))
print(f"Nível de confiança: {confidence_level}")
print(f"Graus de liberdade: {degrees_of_freedon}")
print(f"Intervalo obtido está entre {interval[0]} e {interval[1]}")
print("-"*60)


## Cálculo da proporção de pessoas mais altas do que 1,80 cm

In [ ]:
from scipy.stats import norm

p: float = len(X_sample.loc[X_sample >= 180.0]) / len(X_sample)

def calculate_proportion_variation(n: int, p: float, z_critical: float) -> float:
    return z_critical * ((p * (1 - p)) / n)

z_critical: float = norm.ppf(confidence_level) # type: ignore
prop_var: float = calculate_proportion_variation(len(X_sample), p, z_critical)
interval: tuple[float, float] = (p - prop_var, p + prop_var)

print(" Cálculo da proporção de pessoas maiores que 1,80cm ".center(100, "-"))
print(f"Nível de confiança: {confidence_level}")
print(f"Z critical: {z_critical}")
print(f"A proporção está entre {interval[0]} e {interval[1]}")
print("-"*100)

# Questão F

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

## Pegando os valores dos pontos originais do gráfico

In [ ]:
NOISE: float = 0.4
X: ndarray = np.array((1.0, 1.8, 2.4, 3.0, 3.5, 4.2, 4.8, 5.5, 6.1)).reshape(-1, 1)

def linear_function_with_noise(x: ndarray, a: float = 0.78, b: float = 0.1) -> ndarray:
    return (a*x + b).flatten() + np.random.normal(0,NOISE, size=len(x))

y: ndarray = linear_function_with_noise(X)

## Usando a regressão linear do scikit-learn

A regressão linear do scikit-learn irá encontrar o melhor "a" (`a_optimo`) e o melhor "b" (`b_optimo`), além de calcular o menor SEQ (Soma do erro quadrado)

In [ ]:
model: LinearRegression = LinearRegression()
model.fit(X, y)

b_optimo = model.intercept_
a_optimo = model.coef_[0]
y_pred_opt = model.predict(X)
r2_optimo = r2_score(y, y_pred_opt)

header: str = "---- Melhores parâmetros encontrados pelo Scikit-learn ----"
print(header)
print(f"Intercepto (b): {b_optimo}")
print(f"Inclinação (a): {a_optimo}")
print(f"R-Squared: {r2_optimo}")
print("-" * len(header))

## Utilizando o Statsmodels para conseguir as estatísticas gerais

Ele realizará o mesmo processo que o scikit-learn, porém coleta mais dados para serem analisados

In [ ]:
X_sm = sm.add_constant(X)
model_sm = sm.OLS(y, X_sm).fit()
print(model_sm.summary())

## Definindo as variáveis para guardas o pontos que serão testados

In [ ]:
SLOPE_DIFF: float = 0.5
SLOPES_TO_TEST: int = 6
slopes_to_test = np.linspace(a_optimo - SLOPE_DIFF, a_optimo + SLOPE_DIFF, SLOPES_TO_TEST)

ssr_values = []
r2_values = []
slopes_all = []

## Gerando gráficos

### Gerando o gráfico 1

O gráfico 1 será o gráfico mostrando a reta em si, lá estará a regressão linear e outros 5 a's para serem comparados

### Gerando o gráfico 2

O gráfico 2 é responsável por mostrar a soma do erro quadrado para demonstrar o qual próximo a reta está do resultado ideal

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# ---- Graph 1 ----
# a's tests
ax1.scatter(X, y, color='red', s=100, label='Dados Reais', edgecolor='black')
ax1.plot(X, y_pred_opt, color='black', linewidth=3, label=f'Melhor Ajuste (a={a_optimo})')

print("\n---- Comparação das Rotações ----")

# test others a's
for a_test in slopes_to_test:
    #  y = a*x + b_optimo
    y_manual_pred = a_test * X.flatten() + b_optimo
    
    # metrics
    # Sum Square Residuals (SSR)
    # to plot on graph 2
    ssr = np.sum((y - y_manual_pred) ** 2)
    r2 = r2_score(y, y_manual_pred)
    
    ssr_values.append(ssr)
    r2_values.append(r2)
    slopes_all.append(a_test)
    
    print(f"Slope (a): {a_test} | R2: {r2} | SSR (Erro): {ssr}")
    
    # plot this line on graph 1
    ax1.plot(X, y_manual_pred, linestyle='--', alpha=0.5, label=f'a={a_test}')

ax1.set_title("Variação da Inclinação (Rotação da Reta)")
ax1.legend()
ax1.grid(True, alpha=0.3)

# ---- Graph 2 ----
# specific points
ax2.scatter(slopes_to_test, ssr_values, color='gray', s=50, label='Inclinações Testadas')

# plot optimal point (Scikit-Learn)
ssr_opt = np.sum((y - y_pred_opt) ** 2)
ax2.scatter([a_optimo], [ssr_opt], color='red', s=100, zorder=5, label='Mínimo (Scikit-Learn)')
ax2.annotate('Melhor "a"\n(Menor Erro)', xy=(a_optimo, ssr_opt), xytext=(a_optimo, ssr_opt + 2),
             arrowprops=dict(facecolor='black', shrink=0.05), horizontalalignment='center')

ax2.set_title("Soma dos Quadrados dos Resíduos vs. Inclinação")
ax2.set_xlabel("Valor de 'a' (Inclinação)")
ax2.set_ylabel("Sum of Squared Residuals (SSR)")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Questão G

## Criando a função para a fatoração de LU com pivoteamento

In [ ]:
def lu_factoring_with_pivoting(A_in, b_in):
    A = A_in.copy().astype(float)
    b = b_in.copy().astype(float)
    n = len(A)
    
    # Permutation vector p (p(i) = i)
    # start with [0, 1, ..., n-1]
    p = np.arange(n) 

    print("="*50)
    print("INÍCIO DO PROCESSO (EXEMPLO 7)")
    print(f"Matriz A(0) Inicial:\n{A}")
    print(f"Vetor p inicial: {p + 1}") # +1 apenas para exibição igual ao livro
    print("="*50)

    # --- Step 1: Factoring (Calculate factors L e U) ---
    # Loop k de 1 até n-1 (no Python: 0 até n-2)
    for k in range(n - 1):
        print(f"\n--- ETAPA {k + 1} ---")
        
        # 1. pivot (pv) e pivot position (r)
        # find the largest absolute value in column k, line k until n
        pv = abs(A[k, k])
        r = k
        for i in range(k + 1, n):
            if abs(A[i, k]) > pv:
                pv = abs(A[i, k])
                r = i
        
        print(f"Pivô escolhido: {A[r, k]} (estava na linha {r+1})")

        # check singularity
        if pv == 0:
            print("A matriz é singular!")
            return None

        # 2. permutation (permute lines k and r) if r != k
        if r != k:
            print(f"Permutando linhas {k+1} e {r+1}...")
            aux_p = p[k]
            p[k] = p[r]
            p[r] = aux_p
            
            # swap matrix A line (entirely line)
            row_k = A[k, :].copy()
            row_r = A[r, :].copy()
            A[k, :] = row_r
            A[r, :] = row_k
            
            print(f"Vetor p após troca: {p + 1}")
            print(f"Matriz após troca de linhas:\n{A}")
        else:
            print("Nenhuma troca de linha necessária nesta etapa.")

        # 3. Gaussian elimination and multipliers calculate
        # for i = k+1 until n
        for i in range(k + 1, n):
            # calculate multiplier: m = a(i,k)/a(k,k)
            m = A[i, k] / A[k, k]
            
            # store the multiplier in bottom of the matrix (L)
            A[i, k] = m 
            
            # Atualiza os elementos restantes da linha: a(i,j) = a(i,j) - m*a(k,j)
            # update all line remaining elements
            for j in range(k + 1, n):
                A[i, j] = A[i, j] - m * A[k, j]

        print(f"\nMatriz A({k+1}) após eliminação (multiplicadores na parte inferior):")

        # to improve readability
        print(np.round(A, 4)) 

    # --- Finish Factoring ---
    
    # extrac L and U
    L = np.tril(A, -1) + np.eye(n) # Parte [L]ower triangle + identity
    U = np.triu(A)                 # Parte [U]pper triangle
    
    print("\n" + "="*50)
    print("OS FATORES L e U SÃO:")
    print("Matriz L:")
    print(L)
    print("\nMatriz U:")
    print(U)
    print("="*50)

    # --- Step 2: Solve triangular systems ---
    
    # i) Ly = Pb (onde c = Pb)
    # permutation from b vector to obtain c vector
    c = np.zeros(n)
    print("\nResolvendo Ly = Pb:")
    print(f"Vetor b original: {b}")
    print(f"Aplicando permutações p={p+1} em b...")
    for i in range(n):
        c[i] = b[p[i]]
    print(f"Vetor Pb (lado direito permutado): {c}")

    # Progressive replacement (Ly = c)
    y = np.zeros(n)
    for i in range(n):
        soma = 0
        for j in range(i): # j = 0 until i-1
            soma += L[i, j] * y[j]
        y[i] = c[i] - soma
    
    print(f"Vetor y encontrado: {y}")

    # ii) Ux = y
    # Progressive replacement
    x = np.zeros(n)
    print("\nResolvendo Ux = y:")
    for i in range(n - 1, -1, -1): # i = n-1 until 0 (descending)
        soma = 0
        for j in range(i + 1, n):
            soma += U[i, j] * x[j]
        x[i] = (y[i] - soma) / A[i, i] # A[i,i] is the U diagonal element
    
    print(f"\nSOLUÇÃO FINAL (Vetor x):")
    # column vector
    print(x.reshape(-1, 1))

    return x

## Resolvendo o exemplo 7

In [ ]:
A_ex7 = np.array([
    [3.0, -4.0, 1.0],
    [1.0, 2.0, 2.0],
    [4.0, 0.0, -3.0]
])

b_ex7 = np.array([9.0, 3.0, -2.0])

# Rodar a função
solution = lu_factoring_with_pivoting(A_ex7, b_ex7)

# Questão H

In [ ]:
def theorem_4_converge(A: ndarray) -> bool:
    n: int = len(A)
    alphas: list[float] = []
    
    print("="*60)
    print("TEOREMA 4: VERIFICAÇÃO DO CRITÉRIO DAS LINHAS")
    print("="*60)
    
    # iterate for all lines k
    for k in range(n):
        pivot: float = abs(A[k, k])
        elements_sum: int = 0
        
        # sum line elements, excepts the diagonal element
        terms_list: list[str] = []
        for j in range(n):
            if j != k:
                val = abs(A[k, j])
                elements_sum += val
                terms_list.append(f"{val}")
        
        alpha_k: float = elements_sum / pivot
        alphas.append(alpha_k)
        
        # show the working theorem
        terms_str: str = " + ".join(terms_list)
        print(f"alpha_{k+1} = ({terms_str}) / {pivot} = {elements_sum} / {pivot} = {alpha_k}")

    max_alpha: float = max(alphas)
    print("-" * 60)
    print(f"Máximo alpha = {max_alpha}")
    
    if max_alpha < 1:
        print("Como max_alpha < 1, pelo critério das linhas, temos garantia de convergência.")
        return True
    else:
        print("O critério das linhas não foi satisfeito (não garante convergência, mas pode convergir).")
        return False

## Método Gauss-Jacobi

In [ ]:
def gauss_jacobi(A: ndarray, b: ndarray, x0: ndarray, epsilon: float = 0.05, max_iter: int = 10) -> None:
    n: int = len(b)
    x: ndarray = x0.copy()
    print("\n" + "="*60)
    print("EXEMPLO 10: SOLUÇÃO POR GAUSS-JACOBI")
    print("="*60)
    print(f"x(0) inicial: {x}")
    print(f"Epsilon (erro tolerado): {epsilon}")
    
    # show equations for each iteration
    print("\nO processo iterativo é (Fórmulas isoladas):")
    for i in range(n):
        terms: list[str] = []
        terms.append(f"{b[i]}/{A[i,i]}")
        for j in range(n):
            if i != j:
                coef = -A[i,j]/A[i,i]
                terms.append(f"({coef:+.2f} * x_{j+1})")
        print(f"x_{i+1}^(k+1) = {' '.join(terms)}")

    # iterations loop
    for k in range(max_iter):
        print(f"\n--- Iteração k = {k} (Calculando x^{k+1}) ---")
        
        x_new: ndarray = np.zeros_like(x)
        
        # Gauss-Jacobi formula
        for i in range(n):
            elements_sum = 0
            for j in range(n):
                if i != j:
                    elements_sum += A[i, j] * x[j]
            x_new[i] = (b[i] - elements_sum) / A[i, i]
        
        # relative error (dr)
        # dr = max(|x_new - x_old|) / max(|x_new|)
        diff: ndarray = np.abs(x_new - x)
        max_diff: float = np.max(diff)
        max_x_new: float = np.max(np.abs(x_new))
        dr: float = max_diff / max_x_new
        
        # show parcial results
        print(f"x({k+1}) = {x_new}")
        
        print(f"Distâncias: |x_new - x_old| = {diff}")
        print(f"Máximo numerador (diferença): {max_diff:.4f}")
        print(f"Máximo denominador (valor atual): {max_x_new:.4f}")
        print(f"d({k+1}) = {max_diff:.4f} / {max_x_new:.4f} = {dr:.4f}")
        
        # stop condition
        if dr < epsilon:
            print(f"\nCONVERGIU! O erro {dr:.4f} é menor que epsilon {epsilon}.")
            print(f"Solução final x({k+1}) = \n{x_new.reshape(-1,1)}")
            break
            
        x = x_new # update to the next iteration

## Solucionando o exemplo 10

In [ ]:
# 10x1 + 2x2 +  x3 =  7
#  x1 + 5x2 +  x3 = -8
# 2x1 + 3x2 + 10x3 =  6

A = np.array([
    [10.0, 2.0, 1.0],
    [1.0, 5.0, 1.0],
    [2.0, 3.0, 10.0]
])

b = np.array([7.0, -8.0, 6.0])

# initial aproximation: Exemplo 10
x0 = np.array([0.7, -1.6, 0.6])

# theorem
converge = theorem_4_converge(A)

if converge:
    gauss_jacobi(A, b, x0, epsilon=0.05)

# Questão I

## Criando o critério de linha

In [ ]:
def row_criterion(A: ndarray) -> bool:
    n: int = len(A)
    alphas: list[float] = []
    
    print("="*60)
    print("CRITÉRIO DAS LINHAS (Teorema 4)")
    print("="*60)
    
    for i in range(n):
        # Diagonal element
        pivot = abs(A[i, i])
        
        # Sum of other elements in the row
        row_sum = 0
        sum_str_parts: list[str] = []
        for j in range(n):
            if i != j:
                val: float = abs(A[i, j])
                row_sum += val
                sum_str_parts.append(f"{val}")
        
        alpha: float = row_sum / pivot
        alphas.append(alpha)
        
        # Formatting output like the book
        numerator_str: str = " + ".join(sum_str_parts)
        print(f"alpha_{i+1} = ({numerator_str}) / {pivot} = {alpha:.4f}")

    max_alpha: float = max(alphas)
    print("-" * 30)
    print(f"Máximo alpha = {max_alpha:.4f}")
    
    if max_alpha < 1:
        print("Resultado: O critério das linhas foi satisfeito (Garante convergência).")
        return True
    else:
        print("Resultado: O critério das linhas NÃO foi satisfeito.")
        return False

## Cirando o critério de Sassenfeld

In [ ]:
def sassenfeld_criterion(A: ndarray) -> bool:
    n: int = len(A)
    betas: ndarray = np.zeros(n)
    
    print("\n" + "="*60)
    print("CRITÉRIO DE SASSENFELD")
    print("="*60)
    
    for i in range(n):
        # Denominator is always the diagonal element
        denominator: float = abs(A[i, i])
        
        numerator_val = 0
        numerator_str_parts: list[str] = []
        
        # Loop through columns
        for j in range(n):
            if j != i:
                coeff = abs(A[i, j])
                if j < i:
                    # If j < i, use the beta_j already computed
                    term = coeff * betas[j]
                    numerator_val += term
                    numerator_str_parts.append(f"{coeff}*{betas[j]:.2f}")
                else:
                    # If j > i, use the raw coefficient (assume beta = 1 implicitly in logic)
                    numerator_val += coeff
                    numerator_str_parts.append(f"{coeff}")
        
        beta: float = numerator_val / denominator
        betas[i] = beta
        
        # Formatting output
        numerator_str: str = " + ".join(numerator_str_parts)
        if not numerator_str: numerator_str = "0"
        
        print(f"beta_{i+1} = ({numerator_str}) / {denominator} = {beta:.4f}")
        
    max_beta: float = np.max(betas)
    print("-" * 30)
    print(f"Máximo beta = {max_beta:.4f}")

    if max_beta < 1:
        print("Resultado: O critério de Sassenfeld foi satisfeito (Garante convergência).")
        return True
    else:
        print("Resultado: O critério de Sassenfeld NÃO foi satisfeito.")
        return False

## Código responsável por resolver as equações obtidas

In [ ]:
def gauss_seidel_solver(A: ndarray, b: ndarray, x0: ndarray, epsilon: float = 0.05, max_iter: int = 10) -> None:
    n: int = len(b)
    x: ndarray = x0.copy()
    
    print("\n" + "="*60)
    print(f"MÉTODO DE GAUSS-SEIDEL (Exemplo 14 - Epsilon = {epsilon})")
    print("="*60)
    print(f"x(0) inicial: {x.flatten()}") # Display as flat list
    
    # 1. Print the Iterative Process Equations (Manual extraction for visual match)
    # x1 = (5 - x2 - x3)/5  -> 1 - 0.2x2 - 0.2x3
    # x2 = (6 - 3x1 - x3)/4 -> 1.5 - 0.75x1 - 0.25x3
    # x3 = (0 - 3x1 - 3x2)/6 -> 0 - 0.5x1 - 0.5x2
    print("\nO processo iterativo é:")
    print(f"x1^(k+1) = {b[0]/A[0,0]} - {abs(A[0,1]/A[0,0])}x2^(k) - {abs(A[0,2]/A[0,0])}x3^(k)")
    print(f"x2^(k+1) = {b[1]/A[1,1]} - {abs(A[1,0]/A[1,1])}x1^(k+1) - {abs(A[1,2]/A[1,1])}x3^(k)")
    print(f"x3^(k+1) = {b[2]/A[2,2]} - {abs(A[2,0]/A[2,2])}x1^(k+1) - {abs(A[2,1]/A[2,2])}x2^(k+1)")

    for k in range(max_iter):
        print(f"\n--- Iteração k = {k} ---")
        
        x_old = x.copy()
        
        # Gauss-Seidel Update Loop
        # Note: x is updated in-place, so x[j] for j < i is already the NEW value
        for i in range(n):
            sum_Ax = 0
            calculation_str: list[str] = []
            
            for j in range(n):
                if i != j:
                    sum_Ax += A[i, j] * x[j]
                    
            # Calculate new value
            val = (b[i] - sum_Ax) / A[i, i]
            x[i] = val
            
            # Print specific calculations for the first iteration (k=0) to match book
            if k == 0:
                print(f"x_{i+1}^(1) = {val:.4f} (Calculado usando valores atualizados)")

        # Error Calculation (Distance)
        # formula: dr = max(|x_new - x_old|) / max(|x_new|)
        diff: ndarray = np.abs(x - x_old)
        max_diff: float = np.max(diff)
        max_x_new: ndarray = np.max(np.abs(x))
        dr: ndarray = max_diff / max_x_new
        
        print(f"\nVetor x({k+1}): \n{x.reshape(-1, 1)}")
        
        # Detailed error printing like the book
        print(f"Distâncias |x^(k+1) - x^(k)|: {diff.flatten()}")
        print(f"Máx numerador (diff): {max_diff:.4f}")
        print(f"Máx denominador (norma): {max_x_new:.4f}")
        print(f"d({k+1}) = {max_diff:.4f} / {max_x_new:.4f} = {dr:.4f}")
        
        if dr < epsilon:
            print(f"Conclusão: Como {dr:.4f} < {epsilon}, CONVERGIU.")
            break
        else:
            print(f"Conclusão: Como {dr:.4f} > {epsilon}, continua...")

    print("\n" + "="*60)
    print("SOLUÇÃO FINAL APROXIMADA:")
    print(x.reshape(-1, 1))
    print("="*60)

## Rodando o exemplo 14

In [ ]:
# 5x1 + x2 + x3 = 5
# 3x1 + 4x2 + x3 = 6
# 3x1 + 3x2 + 6x3 = 0

A_ex14: ndarray = np.array([
    [5.0, 1.0, 1.0],
    [3.0, 4.0, 1.0],
    [3.0, 3.0, 6.0]
])

b_ex14: ndarray = np.array([5.0, 6.0, 0.0])

# Initial guess
x0_ex14: ndarray = np.array([0.0, 0.0, 0.0])

# 1. Apply Row Criterion
row_criterion(A_ex14)

# 2. Apply Sassenfeld Criterion
sassenfeld_criterion(A_ex14)

# 3. Solve using Gauss-Seidel
gauss_seidel_solver(A_ex14, b_ex14, x0_ex14, epsilon=5e-2)